In [8]:
import numpy as np 
import tensorflow as tf

import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline

np.random.seed(0)
tf.set_random_seed(0)

In [9]:
# Load MNIST data in a format suited for tensorflow.

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
n_samples = mnist.train.num_examples

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
n_hidden = 500
n_z = 20
batchsize = 100

In [11]:
# leaky reLu unit
def lrelu(x, leak=0.2, name="lrelu"):
    with tf.variable_scope(name):
        f1 = 0.5 * (1 + leak)
        f2 = 0.5 * (1 - leak)
        return f1 * x + f2 * abs(x)

# standard convolution layer
def conv2d(x, inputFeatures, outputFeatures, name):
    with tf.variable_scope(name):
        w = tf.get_variable("w",[5,5,inputFeatures, outputFeatures], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b = tf.get_variable("b",[outputFeatures], initializer=tf.constant_initializer(0.0))
        conv = tf.nn.conv2d(x, w, strides=[1,2,2,1], padding="SAME") + b
        return conv
    
# fully-conected layer
def dense(x, inputFeatures, outputFeatures, scope=None, with_w=False):
    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [inputFeatures, outputFeatures], tf.float32, tf.random_normal_initializer(stddev=0.02))
        bias = tf.get_variable("bias", [outputFeatures], initializer=tf.constant_initializer(0.0))
        if with_w:
            return tf.matmul(x, matrix) + bias, matrix, bias
        else:
            return tf.matmul(x, matrix) + bias

In [16]:
# encoder
def recognition(input_images):
    with tf.variable_scope("recognition"):
        h1 = lrelu(conv2d(input_images, 1, 16, "d_h1")) # 28x28x1 -> 14x14x16
        h2 = lrelu(conv2d(h1, 16, 32, "d_h2")) # 14x14x16 -> 7x7x32
        h2_flat = tf.reshape(h2,[ batchsize, 7*7*32])

        w_mean = dense(h2_flat, 7*7*32, n_z, "w_mean")
        w_stddev = dense(h2_flat, 7*7*32, n_z, "w_stddev")

    return w_mean, w_stddev

In [17]:
images = tf.placeholder(tf.float32, [None, 784])
image_matrix = tf.reshape(images,[-1, 28, 28, 1])
z_mean, z_stddev = recognition(image_matrix)
samples = tf.random_normal([batchsize,n_z],0,1,dtype=tf.float32)
guessed_z = z_mean + (z_stddev * samples)


In [18]:
def conv_transpose(x, outputShape, name):
    with tf.variable_scope(name):
        # h, w, out, in
        w = tf.get_variable("w",[5,5, outputShape[-1], x.get_shape()[-1]], initializer=tf.truncated_normal_initializer(stddev=0.02))
        b = tf.get_variable("b",[outputShape[-1]], initializer=tf.constant_initializer(0.0))
        convt = tf.nn.conv2d_transpose(x, w, output_shape=outputShape, strides=[1,2,2,1])
        return convt


In [20]:
def generation(z):
        with tf.variable_scope("generation"):
            z_develop = dense(z, n_z, 7*7*32, scope='z_matrix')
            z_matrix = tf.nn.relu(tf.reshape(z_develop, [batchsize, 7, 7, 32]))
            h1 = tf.nn.relu(conv_transpose(z_matrix, [batchsize, 14, 14, 16], "g_h1"))
            h2 = conv_transpose(h1, [batchsize, 28, 28, 1], "g_h2")
            h2 = tf.nn.sigmoid(h2)

        return h2

In [21]:
generated_images = generation(guessed_z)
generated_flat = tf.reshape(generated_images, [batchsize, 28*28])


In [ ]:
generation_loss = -tf.reduce_sum(images * tf.log(1e-8 + generated_flat) + (1-images) * tf.log(1e-8 + 1 - generated_flat),1)
latent_loss = 0.5 * tf.reduce_sum(tf.square(z_mean) + tf.square(z_stddev) - tf.log(tf.square(z_stddev)) - 1,1)
cost = tf.reduce_mean(generation_loss + latent_loss)
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
